In [ ]:
import os
import re
import json
import chardet
import scr.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'cp932'
output_encoding = 'utf16'

pf._init_()
name = ''

In [ ]:
# 从游戏原文本文件提取日文

def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in "#*[":
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [ ]:
# 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [ ]:
# 把字典翻译

# print(format_yu_ris('歩「じゃあ≪射精／だ≫して……ね？　いっぱい……≪射精／だ≫してほしい」'))
pf.translate(delete_func=None)
# print(pf._translate('「あーうそうそ！　冗談だってば！\r\n　昼飯は後でちゃんと喰わせるから、\r\n　今は俺を信じてもうちょっと歩こうぜ！」', ))

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:
# 替换文本
def find_all(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in "#*[":
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find_all, encoding, output_encoding)

In [ ]:
data = pf.open_json('intermediate_file/jp_chs.json')
for key in data:
    data[key] = key
pf.save_json('intermediate_file/jp_chs.json', data)

In [ ]:
'重新开始'.encode('gbk')

In [ ]:
a = '''
はい、私がやります！
妻にやらせたいと思います。
お酌
チョンマゲ
裸踊り
二人羽織
お色気にお酌
ピラミッド組体操
ビール・テイスティング
包茎チョンマゲ
お酌で催眠術
泡ビール・イッキ！
女相撲でお着替え
女相撲で土俵入り
全裸踊り
国旗掲揚
ハラ踊り
ストリップ・チアガール
お馬さんごっこ
じゃじゃ馬ならし
公衆トイレ
メッセージボックス
お酌
ピラミッド組体操
二人羽織
お色気にお酌
ビール・テイスティング
裸踊り
チョンマゲ
国旗掲揚
お酌で催眠術
女相撲でお着替え
全裸踊り
ハラ踊り
包茎チョンマゲ
女相撲で土俵入り
ストリップ・チアガール
泡ビール・イッキ！
お馬さんごっこ
じゃじゃ馬ならし
公衆トイレ
'''
print((a.splitlines()))